In [42]:
import pandas as pd
import numpy as np
import pickle
import os
import time
import matplotlib.pyplot as plt
import quandl
from datetime import datetime
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [43]:
#Helper function to get BTC exchange's API data
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from cache'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

In [44]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]

    return pd.DataFrame(series_dict)

In [45]:
btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD')

Cached BCHARTS/KRAKENUSD at BCHARTS-KRAKENUSD.pkl


In [46]:
exchanges = ['COINBASE','BITSTAMP','ITBIT']

exchange_data = {}

exchange_data['KRAKEN'] = btc_usd_price_kraken

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

Cached BCHARTS/COINBASEUSD at BCHARTS-COINBASEUSD.pkl
Cached BCHARTS/BITSTAMPUSD at BCHARTS-BITSTAMPUSD.pkl
Cached BCHARTS/ITBITUSD at BCHARTS-ITBITUSD.pkl


In [47]:
def get_json_data(json_url, cache_path):
    '''Download and cache JSON data, return as a dataframe.'''
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(json_url))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(json_url, cache_path))
    return df

In [48]:
btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')
btc_usd_datasets.replace(0, np.nan, inplace=True)

In [49]:
base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2018-05-01', '%Y-%m-%d') # get data from the start of 2015
#end_date = datetime.strptime('2018-05-02', '%Y-%m-%d') # up until today
end_date = datetime.now()
pediod = 300 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), pediod)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

In [50]:
cryptos = ['ETH','XRP','BCH','STR','LTC','XMR','ETC','ZEC','DOGE','LSK','BTS','SC','XEM','DASH','STRAT','DGB','NXT','ZRX']

crypto_data = {}
crypto_price_df = get_crypto_data('USDT_BTC')
crypto_data['USDT'] = crypto_price_df
for c in cryptos:
    coinpair = 'BTC_{}'.format(c)
    crypto_price_df = get_crypto_data(coinpair)
    crypto_data[c] = crypto_price_df
#crypto_data['BTC']=btc_usd_price_kraken

Cached https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start=1525158000.0&end=1525833083.064413&period=300 at USDT_BTC
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1525158000.0&end=1525833083.064413&period=300 at BTC_ETH
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XRP&start=1525158000.0&end=1525833083.064413&period=300 at BTC_XRP
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_BCH&start=1525158000.0&end=1525833083.064413&period=300 at BTC_BCH
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_STR&start=1525158000.0&end=1525833083.064413&period=300 at BTC_STR
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1525158000.0&end=1525833083.064413&period=300 at BTC_LTC
Cached https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XMR&start=1525158000.0&end=1525833083.064413&period=300 at BTC_X

In [51]:
# Calculate the average BTC price as a new column
btc_usd_datasets['avg_btc_price_usd'] = btc_usd_datasets.mean(axis=1)
#Calculate USD Price as a new column in each crypto dataframe
for c in crypto_data.keys():
    if c != 'USDT':
        crypto_data[c]['price_usd'] =  crypto_data[c]['weightedAverage'] * crypto_data['USDT']['weightedAverage']

In [52]:
#crypto_data['BTC']=btc_usd_datasets
#crypto_data['BTC']['price_usd'] = crypto_data['BTC']['avg_btc_price_usd']

In [53]:
#combined_df = merge_dfs_on_column(list(crypto_data.values()), list(crypto_data.keys()), 'price_usd')

In [54]:
#save all of the cryptocurrency data as individual csv files
for key in crypto_data.keys():
    crypto_data[key].to_csv('{}.csv'.format(key))

In [55]:
crypto_data['LTC'].tail()

,close,high,low,open,quoteVolume,volume,weightedAverage,price_usd
date,,,,,,,,
2018-05-09 02:10:00,0.017259,0.017290,0.017230,0.017233,0.630038,0.010874,0.017259,158.243397
2018-05-09 02:15:00,0.017229,0.017259,0.017229,0.017259,37.069527,0.638711,0.017230,157.763577
2018-05-09 02:20:00,0.017149,0.017253,0.017003,0.017253,739.161864,12.613795,0.017065,155.826489
2018-05-09 02:25:00,0.017120,0.017149,0.017060,0.017149,94.686019,1.616633,0.017074,156.032463
2018-05-09 02:30:00,0.017095,0.017120,0.017095,0.017120,22.008794,0.376290,0.017097,155.978659
